In [2]:
# import the libraries
import requests
from bs4 import BeautifulSoup

## Scraping all pages

In [3]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [4]:
# number of forum pages
num_pages = 2572

In [5]:
# store the comments in a string
all_comments = ""

# go through each page
for page_number in range(1, num_pages + 1):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
    
        # get the html of the page in string form
        page_html = page.text
    
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
    
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
    
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        usermessages = messagelist.find_all(class_ =  "Message userContent")
    
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # a user comment might have multiple <p> tags
            [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            user_comments = usermessages[i].find_all('p')
            for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
                [s.extract() for s in user_comments[j]('a')]
                [s.extract() for s in user_comments[j]('img')]
                [s.extract() for s in user_comments[j]('br')]

            
                # add the users comments to the mega string
                all_comments += user_comments[j].prettify()
    
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete
https://forums.edmunds.com/disc

Page 75 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p76
Page 76 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p77
Page 77 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p78
Page 78 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p79
Page 79 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p80
Page 80 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p81
Page 81 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p82
Page 82 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p83
Page 83 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p84
Page 84 co

Page 149 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p150
Page 150 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p151
Page 151 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p152
Page 152 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p153
Page 153 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p154
Page 154 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p155
Page 155 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p156
Page 156 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p157
Page 157 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

Page 223 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p224
Page 224 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p225
Page 225 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p226
Page 226 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p227
Page 227 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p228
Page 228 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p229
Page 229 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p230
Page 230 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p231
Page 231 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

Page 297 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p298
Page 298 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p299
Page 299 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p300
Page 300 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p301
Page 301 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p302
Page 302 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p303
Page 303 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p304
Page 304 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p305
Page 305 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

...

Page 2260 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2261
Page 2261 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2262
Page 2262 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2263
Page 2263 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2264
Page 2264 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2265
Page 2265 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2266
Page 2266 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2267
Page 2267 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2268
Page 2268 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2332 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2333
Page 2333 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2334
Page 2334 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2335
Page 2335 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2336
Page 2336 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2337
Page 2337 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2338
Page 2338 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2339
Page 2339 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2340
Page 2340 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2404 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2405
Page 2405 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2406
Page 2406 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2407
Page 2407 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2408
Page 2408 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2409
Page 2409 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2410
Page 2410 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2411
Page 2411 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2412
Page 2412 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2476 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2477
Page 2477 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2478
Page 2478 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2479
Page 2479 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2480
Page 2480 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2481
Page 2481 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2482
Page 2482 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2483
Page 2483 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2484
Page 2484 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2548 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2549
Page 2549 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2550
Page 2550 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2551
Page 2551 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2552
Page 2552 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2553
Page 2553 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2554
Page 2554 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2555
Page 2555 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2556
Page 2556 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

In [6]:
print(all_comments[:1000])

<p>
 I'm 2nd post....yee haw!!!!!!!!
</p>
<p>
 Do I get a badge?
</p>
<p>
 Great, but will it be possible to refer to old thread?
</p>
<p>
 And one more thing how do I find and add a discussion to "My Bookmarks?"
</p>
<p>
 If you have posted in that discussion, click on "Participated" and all your past post topics are chronologically listed.
</p>
<p>
 If you are looking for a brand or model or a keyword, use the "Car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names.
</p>
<p>
 if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow.
</p>
<p>
 Thanks
 , it worked like a charm.
</p>
<p>
 Thanks imidazol97. I will try that later but stickguy's approach works for me.
</p>
<p>
 Thanks for that. While this new board loads faster it just seems a little cumbersome and not very user friendly.
</p>
<p>
 No, but it's almost like you're famous.  Just think, 

In [8]:
# let's do some string manipulation
# remove the newline characters, '<p>', '</p','said:'
parsedData = all_comments.replace('\n', '')
parsedData = parsedData.replace(r"<p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"said:", "")
parsedData = parsedData.replace(r":", "")
parsedData = parsedData.lower()

In [10]:
parsedData[:1000]

' i\'m 2nd post....yee haw!!!!!!!! do i get a badge? great, but will it be possible to refer to old thread? and one more thing how do i find and add a discussion to "my bookmarks?" if you have posted in that discussion, click on "participated" and all your past post topics are chronologically listed. if you are looking for a brand or model or a keyword, use the "car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names. if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow. thanks , it worked like a charm. thanks imidazol97. i will try that later but stickguy\'s approach works for me. thanks for that. while this new board loads faster it just seems a little cumbersome and not very user friendly. no, but it\'s almost like you\'re famous.  just think, if this discussion goes on for years and years like the other did, visitors loading this discussion 

In [11]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(parsedData)
new_file.close()

In [12]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    parsedData = parsedData.replace(i[1], i[0])
    if(i[0] not in brands):
        brands.append(i[0])

print('Finished converting model names to brands')
# close the file
file.close()

Finished converting model names to brands


## Word Frequency

In [13]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(parsedData)

In [15]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [21]:
# check
words.most_common()[:100]

[('car', 2209),
 ('one', 987),
 ('like', 944),
 ('would', 907),
 ('get', 730),
 ('new', 662),
 ('think', 568),
 ('good', 511),
 ('back', 448),
 ('time', 446),
 ('know', 429),
 ('years', 416),
 ('dealer', 409),
 ('benz', 405),
 ('could', 401),
 ('much', 395),
 ('people', 389),
 ('ford', 378),
 ('buy', 369),
 ('well', 367),
 ('even', 358),
 ('see', 346),
 ('got', 330),
 ('year', 325),
 ('want', 320),
 ('go', 318),
 ('said', 316),
 ('really', 315),
 ('way', 301),
 ('going', 293),
 ('price', 287),
 ('make', 283),
 ('still', 269),
 ('better', 269),
 ('right', 266),
 ('never', 261),
 ('many', 260),
 ('also', 259),
 ('used', 249),
 ('bmw', 246),
 ('money', 241),
 ('take', 236),
 ('last', 231),
 ('day', 229),
 ('lot', 227),
 ('nice', 227),
 ('old', 225),
 ('first', 225),
 ('gm', 225),
 ('sure', 224),
 ('two', 219),
 ('drive', 218),
 ('problem', 214),
 ('may', 210),
 ('every', 210),
 ('something', 206),
 ('us', 204),
 ('mercedes', 202),
 ('miles', 201),
 ('probably', 197),
 ('use', 195),
 ('loo

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [17]:
# store the stop words
stopwords = stopwords.words('english')[:]

In [18]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [20]:
# check
words.most_common()[:1000]

[('car', 2209),
 ('one', 987),
 ('like', 944),
 ('would', 907),
 ('get', 730),
 ('new', 662),
 ('think', 568),
 ('good', 511),
 ('back', 448),
 ('time', 446),
 ('know', 429),
 ('years', 416),
 ('dealer', 409),
 ('benz', 405),
 ('could', 401),
 ('much', 395),
 ('people', 389),
 ('ford', 378),
 ('buy', 369),
 ('well', 367),
 ('even', 358),
 ('see', 346),
 ('got', 330),
 ('year', 325),
 ('want', 320),
 ('go', 318),
 ('said', 316),
 ('really', 315),
 ('way', 301),
 ('going', 293),
 ('price', 287),
 ('make', 283),
 ('still', 269),
 ('better', 269),
 ('right', 266),
 ('never', 261),
 ('many', 260),
 ('also', 259),
 ('used', 249),
 ('bmw', 246),
 ('money', 241),
 ('take', 236),
 ('last', 231),
 ('day', 229),
 ('lot', 227),
 ('nice', 227),
 ('old', 225),
 ('first', 225),
 ('gm', 225),
 ('sure', 224),
 ('two', 219),
 ('drive', 218),
 ('problem', 214),
 ('may', 210),
 ('every', 210),
 ('something', 206),
 ('us', 204),
 ('mercedes', 202),
 ('miles', 201),
 ('probably', 197),
 ('use', 195),
 ('loo

In [86]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in brands)

In [87]:
brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'car',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'hyundai,',
 'hyundai.',
 'infiniti',
 'kia',
 'kia,',
 'kia.',
 'lincoln',
 'mazda',
 'mercedes',
 'mercedes benz',
 'mercedes-benz',
 'mercury',
 'mitsubishi',
 'nissan',
 'nissan.',
 'pontiac',
 'problem',
 'saturn',
 'seat',
 'sedan',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [88]:
# check
words.most_common()[:]

[('car', 1860),
 ('ford', 342),
 ('bmw', 229),
 ('problem', 189),
 ('mercedes', 149),
 ('chevrolet', 148),
 ('buick', 136),
 ('audi', 129),
 ('volkswagen', 125),
 ('honda', 105),
 ('toyota', 102),
 ('seat', 100),
 ('cadillac', 81),
 ('dodge', 80),
 ('chrysler', 77),
 ('acura', 67),
 ('subaru', 57),
 ('nissan', 55),
 ('lincoln', 54),
 ('hyundai', 49),
 ('mazda', 44),
 ('sedan', 40),
 ('volvo', 39),
 ('kia', 19),
 ('saturn', 8),
 ('pontiac', 7),
 ('infiniti', 6),
 ('mitsubishi', 3),
 ('mercury', 2)]